
## Chuẩn bị dữ liệu


In [4]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as dsp
import numpy as np
import os
import shutil
import random
from hmmlearn.base import ConvergenceMonitor
from hmmlearn import hmm
from sklearn import preprocessing
from scipy.io.wavfile import read

Ta sẽ thử nghiệm với khẩu lệnh cần nhận dạng là  ***nhay***

In [20]:
!ls

Dataset                                hmm.ipynb
dataset_dtw                            mfcc-implementation-and-tutorial.ipynb
dtw.ipynb                              mfcc.ipynb
dtw_v1.ipynb


### Chuẩn bị tập train

In [59]:
path1 = "./Dataset/{}/{}/file{}.wav"
path2 = "./dataset_dtw/train/{}/file{}.wav"

names = ["nghia","khiem","huy","hoang","long"]
labels = ["a", 'b', 'ban','nhay','xuong','len','phai','trai']
for label in labels:
    count = 0
    for name in names:
        array = random.sample(range(1,20),3)
        for index in array:
            count += 1
            try:
                shutil.copy(path1.format(name, label,index), path2.format(label,count))
            except:
                pass

### Chuẩn bị tập test

In [61]:
path1 = "./Dataset/{}/{}/file{}.wav"
path2 = "./dataset_dtw/test/{}/file{}.wav"

names = ["nghia","khiem","huy","hoang","long"]
labels = ["a", 'b', 'ban','nhay','xuong','len','phai','trai']
for label in labels:
    count = 0
    for name in names:
        array = random.sample(range(20,30),3)
        for index in array:
            count += 1
            try:
                shutil.copy(path1.format(name, label,index), path2.format(label,count))
            except:
                pass

In [63]:
path = './dataset_dtw/train/nhay/file{}.wav'

In [64]:
p1 = path.format('1')
p2 = path.format('2')
p3 = path.format('3')

In [65]:
dsp.Audio(p1)

In [66]:
dsp.Audio(p2)

In [67]:
dsp.Audio(p3)

## Trích xuất đặc trưng

Ta sẽ xây dựng hàm **extract_fea** để trích xuất đặc trưng MFCC, delta (đạo hàm bậc 1), deltadelta (đạo hàm bậc 2) của từng đoạn âm thanh sau đó kết hợp lại để thu được đặc trưng đại diện cho mỗi âm thanh đó. Số chiều sau khi concat là 20*3=60. Ouput của đặc trưng là (60,x)

In [83]:
def extract_fea(path):
    sound, _ = librosa.load(path)
    mfcc   = librosa.feature.mfcc(y=sound)
    delta1  = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)
    ft = np.concatenate((mfcc, delta1, delta2))
    
    return ft

In [84]:
ft1 = extract_fea(p1)
ft2 = extract_fea(p2)
ft3 = extract_fea(p3)

In [85]:
ft1.shape

(60, 21)

In [86]:
def normalize(feature):
    normalized = np.full_like(feature, 0)
    for i in range(feature.shape[1]):
        # đưa giá trị trung bình về 0
        normalized[:,i] = feature[:,i] - np.mean(feature[:,i])
        # đưa giá trị về khoảng [-1, 1]
        normalized[:,i] = normalized[:,i] / np.max(np.abs(normalized[:,i])) 
    return normalized

In [87]:
template0 = normalize(ft1)
template1 = normalize(ft2)
template2 = normalize(ft3)

In [88]:
template0.shape

(60, 21)


## Xây dựng template trung bình


Thực hiện dóng hàng template1 và template2 và chọn template 0 là template chuẩn sử dụng sequence.dtw của librosa

In [89]:
_, align1 = librosa.sequence.dtw(template0, template1)
_, align2 = librosa.sequence.dtw(template0, template2)

In [90]:
align1.shape

(24, 2)

In [122]:
list_temp = [template0, template1, template2]
list_align = [align1, align2]

In [123]:
def build_average_template(list_template, list_align):
    count = np.ones(len(list_template[0][0])) 
    summ = list_template[0].copy() 
    for m, n in list_align[0]:
        count[m] += 1
        summ[:,m] += list_template[1][:,n]
    for m, n in list_align[1]:
        count[m] += 1
        summ[:,m] += list_template[2][:,n]
    average_template = summ / count

    return average_template

In [124]:
average_template = build_average_template(list_temp, list_align)
average_template.shape 

(60, 21)


## Nhận diện khẩu lệnh 


Ta sẽ so sánh khẩu lệnh trung bình của 3 khẩu lệnh **nhay** với các khẩu lệnh khác để nhận diện đâu là khẩu lệnh giống với khẩu lệnh trung bình

In [105]:
command_list = ['a', 'b', 'nhay', 'len', 'xuong', 'trai', 'phai', 'ban']
label_list = ['a', 'b', 'nhảy', 'lên', 'xuống', 'trái', 'phải', 'bắn']

In [101]:
test_path = './dataset_dtw/test/{}/file1.wav'

In [102]:
dsp.Audio(test_path.format('a'))

In [103]:
def build_template(path):
    ft = extract_fea(path)
    norm_ft = normalize(ft)
    return norm_ft

Tính "khoảng cách" giữa template của các khẩu lệnh và template trung bình. Giá trị càng nhỏ thì 2 template càng giống nhau

In [106]:
score = [librosa.sequence.dtw(build_template(test_path.format(cmd)), average_template)[0][-1,-1] for cmd in command_list]


## Kết quả thử nghiệm


In [107]:
label_list[np.argmin(score)]

'nhảy'

Ta thấy template trung bình được tạo từ 3 âm thanh **nhảy** gần với template test **nhảy** nhất 

## Demo

Ta sẽ xây dựng template trung bình của tất cả các khẩu lệnh và test demo

In [125]:
def average_template_list(command):
    p_train = ['./dataset_dtw/train/{}/file{}.wav'.format(command, i) for i in range(1, 4)]
    templates = [build_template(p) for p in p_train]
    _, align1 = librosa.sequence.dtw(X=templates[0], Y=templates[1])
    _, align2 = librosa.sequence.dtw(X=templates[0], Y=templates[2])
    l_temp = [templates[0], templates[1], templates[2]]
    l_align = [align1, align2]
    
    average_temp = build_average_template(l_temp, l_align) 

    return average_temp

templates_list = [average_template_list(cmd) for cmd in command_list]

def predict(path, templates_list):
    score = [librosa.sequence.dtw(build_template(path), templates_list[i])[0][-1,-1] for i in range(len(templates))]
    return label_list[np.argmin(score)]

In [209]:
test_path = "./dataset_dtw/test/{}/file{}.wav"

# names = ["nghia","khiem","huy","hoang","long"]
labels = ["a", 'b', 'ban','nhay','xuong','len','phai','trai']
index_nums = random.randint(0,20)
index_labels = random.randint(0,7)
t_path = test_path.format(labels[index_labels], index_nums)
print('Test path file', t_path)

Test path file ./dataset_dtw/test/xuong/file3.wav


In [210]:
dsp.Audio(t_path, rate=22050)

In [211]:
predict(t_path, templates_list)

'xuống'